In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split 
import cv2
from scipy import ndimage  

In [2]:
class Network: 

    def __init__(self, layers=5, neuron=5):
        self.layers = layers 
        self.neuron = neuron 

    def set_train(self, train_data, targets):
        self.x_train, self.y_train = train_data[0], train_data[1]
        self.train_data =  [(self.x_train[i,:,:], self.y_train[:,i]) for i in range(np.shape(self.x_train)[0])] 

        # Initialize random weights. Returns a list containing: 
        # Mx(neuron) 2D array in the first index, where M is the number of initial neurons, and (neuron)xN, where N is the number of neurons in the output layer, and (neuron)x(neuron) in the middle layers.
        self.w_ = [np.random.rand(self.neuron, self.neuron) for i in range(self.layers-1)]
        input_w, output_w = np.random.rand(self.neuron, len(self.x_train[0].flatten())), np.random.rand(targets, self.neuron)
        self.w_.insert(0, input_w)
        self.w_.append(output_w)

        # Initialize random column bias vectors. Returns a list containing: 
        # 1xM row vector in the first index, where M is the number of initial neurons, and 1xN, where N is the number of neurons in the output layer, and 1x(neuron) in the middle layers.
        self.bias_ = [np.random.rand(self.neuron, 1) for i in range(self.layers)]
        input_b, output_b = np.zeros((len(self.x_train[0].flatten()),1) ), np.random.rand(targets,1)
        self.bias_.insert(0, input_b)
        self.bias_.append(output_b)

        return self
    
    def feedforward(self, input):
        input = np.column_stack([np.array(input)])
        initial_act=  input + np.array(self.bias_[0])
        self.activations_ = [initial_act] 
        self.z_ = [np.array(input)]
        for l in range(0,self.layers+1): #Loop over l = 0,1,2,...,L-1 where L-1 is the output layer 
            z = np.matmul(self.w_[l], np.column_stack([np.array(self.activations_[l])])) + self.bias_[l+1]
            self.z_.append(z)
            self.activations_.append(self.threshold(z))
        return self
    
    def output_error(self, output):
        err = (self.activations_[-1] - np.column_stack([output]))*self.threshold_der(self.z_[-1])
        return err 
    
    def cost(self, output):
        return (0.5)*np.linalg.norm(self.activations_[-1] - output)**2
    
    def backpropagate(self, output):
        self.error_array = [self.output_error(output)]
        for l in range(-1, -self.layers-1, -1):
            error_vector = np.dot(np.transpose(np.array(self.w_[l])), np.array(self.error_array[l]))*np.column_stack([np.array(self.threshold_der(self.z_[l-1]))])
            self.error_array.insert(0, error_vector)
        return self
    
    def threshold(self, z): #Hyperbolic tangent
        return np.tanh(z)

    def threshold_der(self, z): #Derivative of the activation function 
        return 1 + self.threshold(z)**2

    
    def train(self, test, epochs = 10, eta = 0.01, batch_size=100): 
        for iter in range(epochs+1): 
            if test != None:
                print('Epoch {a}/{b}: {c}'.format(a=iter, b= epochs, c=self.evaluate(test)))
            np.random.shuffle(self.train_data) 
            mini_batches = [self.train_data[k:k+batch_size] for k in range(0,len(self.train_data), batch_size)]
            for mini_batch in mini_batches:
                for (m,n) in mini_batch:
                    self.feedforward(np.array(m.flatten()))
                    self.backpropagate(n) 
                    for l in range(-1, -self.layers-2, -1): 
                        update_w = np.dot(self.error_array[l], self.activations_[l-1].T)
                        update_b = self.error_array[l] 
                        self.w_[l] -= update_w*(eta/batch_size)
                        self.bias_[l] -= update_b*(eta/batch_size)
        return self.w_, self.bias_
    
    def evaluate(self, test_data): 
        i = 0
        for (input,output) in test_data:
            input = np.array(input).flatten()
            input = np.column_stack([np.array(input)])
            a_list = [input] 
            z_list = [input]
            for l in range(0,self.layers+1):
                z = np.matmul(self.w_[l], np.column_stack([np.array(a_list[l])])) + self.bias_[l+1]
                z_list.append(z)
                a_list.append(self.threshold(z))
            if np.argmax(a_list[-1]) == output:
                i += 1
        return i/len(test_data)
            
        


In [3]:
import os
from PIL import Image

# Get the current working directory
current_dir = os.getcwd()

# Define the path to the separate directory
galaxy_path = r'C:\Users\Ron\Documents\ML Datasets\archive\Cutout Files\galaxy'
star_path = r'C:\Users\Ron\Documents\ML Datasets\archive\Cutout Files\star'
galaxies= os.listdir(galaxy_path)
stars = os.listdir(star_path)

galaxy_data = [cv2.imread(os.path.join(galaxy_path, galaxy),0) for galaxy in galaxies]
star_data = [cv2.imread(os.path.join(star_path, star),0) for star in stars]
np.random.shuffle(galaxy_data)
np.random.shuffle(star_data)



In [4]:
N_galaxy = len(galaxy_data)
N_star = len(star_data)

galaxy_ytrain = np.zeros((2,3*N_galaxy//4))
galaxy_ytrain[0] = 1 
star_ytrain = np.zeros((2,3*N_star//4)) 
star_ytrain[1] = 1

galaxy_ytest = np.zeros((2, N_galaxy - 3*N_galaxy//4))
galaxy_ytest[0] = 1 
star_ytest = np.zeros((2,N_star - 3*N_star//4)) 
star_ytest[1] = 1

In [5]:
train_X= np.array([galaxy_data[i] for i in range(3*len(galaxy_data)//4)] + [star_data[i] for i in range(3*len(star_data)//4)])
train_Y= np.concatenate((galaxy_ytrain, star_ytrain),axis=1)
test_X= np.array([galaxy_data[i] for i in range(3*len(galaxy_data)//4, len(galaxy_data))] + [star_data[i] for i in range(3*len(star_data)//4 , len(star_data))])
test_Y= np.concatenate((galaxy_ytest, star_ytest),axis=1)


In [6]:
train_Y 

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [7]:
test_data_tuple= [(test_X[i], int(test_Y[0][i])) for i in range(len(test_X))]
np.random.shuffle(test_data_tuple)

In [8]:
len(test_data_tuple)

997

In [9]:
net = Network(2,100)
net.set_train((train_X, train_Y), 2) 
net.train(epochs = 20, eta = 10, batch_size=100, test=test_data_tuple)

Epoch 0/20: 0.7632898696088265
Epoch 1/20: 0.5586760280842528
Epoch 2/20: 0.23671013039117353
Epoch 3/20: 0.23671013039117353
Epoch 4/20: 0.23671013039117353
Epoch 5/20: 0.2567703109327984
Epoch 6/20: 0.23771313941825475
Epoch 7/20: 0.6680040120361084
Epoch 8/20: 0.7622868605817452
Epoch 9/20: 0.4242728184553661
Epoch 10/20: 0.23671013039117353
Epoch 11/20: 0.7432296890672017
Epoch 12/20: 0.238716148445336
Epoch 13/20: 0.23671013039117353
Epoch 14/20: 0.7622868605817452
Epoch 15/20: 0.7642928786359077
Epoch 16/20: 0.23570712136409228
Epoch 17/20: 0.6820461384152458
Epoch 18/20: 0.23671013039117353
Epoch 19/20: 0.5767301905717152
Epoch 20/20: 0.23671013039117353


([array([[-105546.43197099,  149639.17186397,   46431.20072175, ...,
           -19825.50812855,   13960.93998222,  -96645.45297606],
         [ -46395.30639413,  -88239.24201992,   22747.71599061, ...,
            38434.52914252,   30939.3348119 ,   75462.23632343],
         [ -63491.17546216,   11153.76969403,  -88237.97999211, ...,
           -24629.0830557 ,   83479.78484238,  -64217.56656562],
         ...,
         [  45805.1387162 ,   -2117.9305819 ,   33022.15487135, ...,
             3562.87190841, -206834.07862066,  -92932.19052546],
         [-127231.99001632,   37828.77164208,  -37175.18515453, ...,
            90468.31070021,   52810.81613418,  -84525.58563602],
         [ -10591.01121464,   32125.51780029,   65741.66962907, ...,
           -13168.28509317, -218498.68645899,   26777.46500224]]),
  array([[ 1.93465917,  0.93493007,  1.94515354, ...,  1.96068267,
          -1.1081506 ,  0.28603005],
         [ 1.20584333, -0.51592535, -0.75673073, ...,  0.60874511,
         

In [10]:
net.evaluate(test_data_tuple)

0.6589769307923772